# Meta Prompting Agent
Meta prompting is a method proposed in the paper "Meta-Prompting: Enhancing Language Models with Task-Agnostic Scaffolding". Given a query, a meta expert generates specific instructions for different experts sequentially. The meta expert will solve the query according to the responses of the experts. Refer to details in the original [paper](https://arxiv.org/abs/2401.12954).

We enhanced the original method by equipping the generated expert with a code interpreter (in this case a UserProxyAgent). Instead of resolving to a dedicated expert Python for programming-related problems, the expert can now generate and execute python code at its will.

We introduce two ways of using MetaPromptAgent.
1. Use a UserProxy to directly initiate chat with MetaPromptAgent.
2. Use MetaAgent from Autobuild.

## Method 1: Directly Initiate Chat

Preparations: Save your api key and endpoint into file named `OAI_CONFIG_LIST`. The config list should look like this:

```python
config_list = [
    {
        'model': 'gpt-4-1106-preview',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    }
]
```

In [3]:
import autogen
from autogen.agentchat.contrib.meta_prompt_agent import MetaPromptAgent
from autogen.agentchat import UserProxyAgent
from autogen.agentchat.contrib.meta_agent import MetaAgent
from autogen.agentchat.contrib.meta_user_proxy_agent import MetaUserProxyAgent

In [4]:
proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: ">> FINAL ANSWER" in x.get("content", ""),
    default_auto_reply="TERMINATE",
)

meta = MetaPromptAgent(
    name="MetaAgent",
    llm_config={
        "config_list": autogen.config_list_from_json("OAI_CONFIG_LIST", filter_dict={"model": ["gpt-4-1106-preview"]}),
    },
    is_termination_msg=lambda x: x.get("content", "").strip().endswith("TERMINATE"),
    code_execution_config={
        "use_docker": False,
    },
)

/workspaces/autogen-autobuild-dev/autogen/agentchat/user_proxy_agent.py:83: UserWarning: Using None to signal a default code_execution_config is deprecated. Use {} to use default or False to disable code execution.
  super().__init__(


In [5]:
proxy.initiate_chat(
    meta,
    message="What is the result when the greatest common factor of 6432 and 132 is increased by 11? Put your final answer in \\box{}.",
)

UserProxy (to MetaAgent):

What is the result when the greatest common factor of 6432 and 132 is increased by 11? Put your final answer in \box{}.

--------------------------------------------------------------------------------
Below is the inner conversation of the MetaPromptAgent
ROUND 0 Meta model output:

To solve the problem of finding the greatest common factor (GCF) of 6432 and 132 and then increasing the result by 11, I will consult the following experts:

1. Expert Mathematician: To calculate the GCF of the two numbers.
2. Expert Verifier: To verify the calculated GCF.
3. Expert Calculator: To add 11 to the verified GCF.

I will begin by consulting the Expert Mathematician to compute the GCF of 6432 and 132.

Expert Mathematician:
"""
You are a mathematics expert, specializing in number theory. Determine the greatest common factor (GCF) of the numbers 6432 and 132.
"""

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...

Response from Expert Mathematician:

Cert

ChatResult(chat_id=None, chat_history=[{'content': 'What is the result when the greatest common factor of 6432 and 132 is increased by 11? Put your final answer in \\box{}.', 'role': 'assistant'}, {'content': 'The Expert Verifier has confirmed the GCF of 6432 and 132 as 12 using a different method, which corroborates the solution provided by the Expert Mathematician. With this confirmation, we can conclude that the GCF is indeed 12. The next step is to increase this value by 11 to answer the original question.\n\nSo, the final answer can be calculated as follows:\n\nFinal GCF = 12\nIncrease by 11: 12 + 11 = 23\n\nSince the solution has been verified by two independent experts, and it matches with what I would expect as well, I can confidently present the final answer:\n\n>> FINAL ANSWER:\n"""\n\\(\\boxed{23}\\)\n"""', 'role': 'user'}], summary='The Expert Verifier has confirmed the GCF of 6432 and 132 as 12 using a different method, which corroborates the solution provided by the Exper

## Method 2: MetaPrompt as a Tool
MetaPrompt is integrated into AutoBuild and can be leveraged as a tool.

In [7]:
general_llm_config = {
    "config_list": autogen.config_list_from_json("OAI_CONFIG_LIST", filter_dict={"model": ["gpt-4-1106-preview"]}),
}
nested_mode_config = {"meta_prompting_llm_config": general_llm_config.copy()}

Set nested_mode to 'meta_prompting' to enable MetaPrompting as a Tool.

In [8]:
meta_agent = MetaAgent(name="meta_agent", llm_config=general_llm_config, nested_mode="meta_prompting")
meta_user_proxy = MetaUserProxyAgent(
    name="meta_user_proxy",
    nested_mode_config=nested_mode_config,
    code_execution_config={
        "use_docker": False,
    },
    max_consecutive_auto_reply=1,
)

In [9]:
meta_user_proxy.initiate_chat(
    meta_agent,
    message="What is the result when the greatest common factor of 6432 and 132 is increased by 11? Put your final answer in \\box{}.",
)

meta_user_proxy (to meta_agent):

What is the result when the greatest common factor of 6432 and 132 is increased by 11? Put your final answer in \box{}.

--------------------------------------------------------------------------------
meta_agent (to meta_user_proxy):

***** Suggested tool Call (call_MoWklx8TDBwcdYjsKoB34D3b): meta_prompting *****
Arguments: 
{"task":"What is the greatest common factor of 6432 and 132?"}
*******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION meta_prompting...


Running meta prompting...
Below is the inner conversation of the MetaPromptAgent
ROUND 0 Meta model output:

Experts I may want to consult for finding the greatest common factor (GCF) of two numbers are:

1. Expert Mathematician: They can apply mathematical theories and techniques to determine the GCF.
2. Expert Number Theorist: As a specialist in number theory, they can focus on the properties and relationships of integers, which is useful for finding the GCF.
3. Expert Calculator: They can perform the calculations required to determine the GCF if a specific algorithm is needed.

Since finding the GCF is a mathematical problem that typically doesn't require multiple steps of verification and can be straightforwardly solved, I will start by contacting the Expert Mathematician to compute the GCF of 6432 and 132.

Expert Mathematician:
"""
You are a mathematics expert with a deep understanding of number theory and arithmetic operations. Please compute the greatest common factor of the nu

ChatResult(chat_id=None, chat_history=[{'content': 'What is the result when the greatest common factor of 6432 and 132 is increased by 11? Put your final answer in \\box{}.', 'role': 'assistant'}, {'tool_calls': [{'id': 'call_MoWklx8TDBwcdYjsKoB34D3b', 'function': {'arguments': '{"task":"What is the greatest common factor of 6432 and 132?"}', 'name': 'meta_prompting'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'The output from the second Expert Mathematician independently confirms the solution provided by the first Expert Mathematician: the greatest common factor (GCF) of 6432 and 132 is 12.\n\nGiven that we have now two independent confirmations of the GCF being 12, and the process used by both experts aligns with a known and reliable mathematical algorithm (the Euclidean algorithm), we have a high level of confidence in the accuracy of the solution.\n\nThus, we can proceed to present the final answer.\n\n>> FINAL ANSWER:\n"""\nThe greatest common factor